In [1]:
# Import libraries
import stanza
import pandas as pd
import string
import re
from tqdm import tqdm
from pandarallel import pandarallel

In [2]:
stanza.download('en')
pandarallel.initialize()

2020-08-14 12:00:04 INFO: Downloading default packages for language: en (English)...
2020-08-14 12:00:04 INFO: File exists: C:\Users\shoeb\stanza_resources\en\default.zip.
2020-08-14 12:00:08 INFO: Finished downloading models and saved to C:\Users\shoeb\stanza_resources.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
nlp = stanza.Pipeline('en')

2020-08-14 12:00:08 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2020-08-14 12:00:08 INFO: Use device: gpu
2020-08-14 12:00:08 INFO: Loading: tokenize
2020-08-14 12:00:12 INFO: Loading: pos
2020-08-14 12:00:12 INFO: Loading: lemma
2020-08-14 12:00:12 INFO: Loading: depparse
2020-08-14 12:00:14 INFO: Loading: sentiment
2020-08-14 12:00:15 INFO: Loading: ner
2020-08-14 12:00:15 INFO: Done loading processors!


In [4]:
df = pd.read_csv("./data/english.txt", sep="\t", header=0, names=["id", "sentence"])

In [5]:
df.head()

,id,sentence
0,2,You would be a great client for Southern India...
1,3,"He believes the 21st century will be the ""cent..."
2,4,They even call the civil rights organization a...
3,5,But while VRE is not a threat to healthy indiv...
4,6,"Earlier on Tuesday, roadside bombs, including ..."


In [41]:
def processLemmatization(text):
    word = ""
    lemma = ""
    punc= set(string.punctuation)
    doc = nlp(text)
    for sent in doc.sentences:
        for wrd in sent.words:
            c = str(wrd.text)
            d = str(wrd.lemma)
            if any(char in punc for char in d)==True:
                word = word.lstrip()
                lemma = lemma.lstrip()
                word += c + " "
                lemma += d + " "
            else:
                word += c + " "
                lemma += d + " "  
    #return a dataframe
    return word, lemma
    # return lemma

In [42]:
parsed_text = {'sentence':[], 'lemma':[]}
for i in tqdm(range(len(df))):
    word, lemma = processLemmatization(df['sentence'][i])
    #extract text and lemma
    parsed_text['sentence'].append(word.rstrip())
    parsed_text['lemma'].append(lemma.rstrip())
text = pd.DataFrame(parsed_text)
text.to_csv(r'./data/eng-lem.txt', sep='\t', index = False)

100%|██████████| 9157/9157 [12:36<00:00, 12.10it/s]
